<a href="https://colab.research.google.com/github/SiaMahmoudi/Object-Detector-in-TensorFlow-Using-Rotated-Bounding-box-Regression/blob/main/Rotated_Bounding_box_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import math as m
from math import pi, cos, sin

In [2]:
SKIP_NEGATIVES = True
NEGATIVE_CLASS = "No-Stair"

In [3]:
from tensorflow import keras


In [4]:
def xml_to_csv(path, skipNegatives):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        if root.find('object'):
            for member in root.findall('object'):
                bbx = member.find('robndbox')                
                cx = round(float(bbx.find('cx').text))
                cy = round(float(bbx.find('cy').text))
                w = round(float(bbx.find('w').text))
                h = round(float(bbx.find('h').text))
                angel = float(bbx.find('angle').text)
                label = member.find('name').text
                value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     label,
                     cx,
                     cy,
                     w,
                     h,
                     m.cos(angel),
                     m.sin(angel)
                     )
                print(value)
                xml_list.append(value)
        elif not skipNegatives:
            value = (root.find('filename').text,
                        int(root.find('size')[0].text),
                        int(root.find('size')[1].text),
                        NEGATIVE_CLASS,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0
                        )
            print(value)
            xml_list.append(value)

    column_name = ['filename', 'width', 'height',
                   'class', 'cx', 'cy', 'w', 'h', 'cos', 'sin']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [5]:
def main():
    datasets = ['Training', 'Validation']
    
    for ds in datasets:
        image_path = os.path.join(paths['Images'], ds)
        xml_df = xml_to_csv(image_path, SKIP_NEGATIVES)
        xml_df.to_csv('Data/{}_data.csv'.format(ds), index=None)
        print('Successfully converted xml to csv.')
        
   

In [6]:
paths = {
    'Images': os.path.join('Images'),
    'Data': os.path.join('Data')
    #'Training': os.path.join('Images','Training'),
   # 'Validation': os.path.join('Images','Validation')
 }

In [7]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
!unzip '/content/Images/ALL.zip' -d '/content/Images'

In [ ]:
import os
main()

In [10]:
TRAINING_CSV_FILE = '/content/Data/Training_data.csv'
TRAINING_IMAGE_DIR = '/content/Images/Training'

Validation_CSV_FILE = '/content/Data/Validation_data.csv'
Validation_IMAGE_DIR = '/content/Images/Validation'

In [11]:
training_image_records = pd.read_csv(TRAINING_CSV_FILE)

train_image_path = os.path.join(os.getcwd(), TRAINING_IMAGE_DIR)


In [12]:
train_images = []
train_targets = []
train_labels = []

In [13]:
classes = ["UpStair", "DownStair"]
width = 350
height = 400
num_classes = 2




In [14]:
for index, row in training_image_records.iterrows():
    
    (filename, width, height, class_name, cx, cy, w, h, cos, sin) = row
    
    train_image_fullpath = os.path.join(train_image_path, filename) + ".jpg"
    train_img = keras.preprocessing.image.load_img(train_image_fullpath, target_size=(height, width))
    train_img_arr = keras.preprocessing.image.img_to_array(train_img)
    
    
    cx = round(cx/ width, 2)
    cy = round(cy/ height, 2)
    w = round(w/ width, 2)
    h = round(h/ height, 2)
    
    train_images.append(train_img_arr)
    train_targets.append((cx, cy, w, h, cos, sin))
    train_labels.append(classes.index(class_name))

In [15]:
Valid_images = []
Valid_targets = []
Valid_labels = []

In [16]:
Validation_image_records = pd.read_csv(Validation_CSV_FILE)

Valid_image_path = os.path.join(os.getcwd(), Validation_IMAGE_DIR)

In [17]:
for index, row in Validation_image_records.iterrows():
    
    (filename, width, height, class_name, cx, cy, w, h, cos, sin) = row
    
    Valid_image_fullpath = os.path.join(Valid_image_path, filename) + ".jpg"
    Valid_img = keras.preprocessing.image.load_img(Valid_image_fullpath, target_size=(height, width))
    Valid_img_arr = keras.preprocessing.image.img_to_array(Valid_img)
    
    
    cx = round(cx/ width, 2)
    cy = round(cy/ height, 2)
    w = round(w/ width, 2)
    h = round(h/ height, 2)
    
    Valid_images.append(Valid_img_arr)
    Valid_targets.append((cx, cy, w, h, cos, sin))
    Valid_labels.append(classes.index(class_name))

In [18]:
train_images = np.array(train_images)
train_targets = np.array(train_targets)
train_labels = np.array(train_labels)

validation_images = np.array(Valid_images)
validation_targets = np.array(Valid_targets)
validation_labels = np.array(Valid_labels)

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import pandas as pd
from PIL import Image 
from PIL.ImageDraw import Draw

In [20]:
#create the common input layer
input_shape = (height, width, 3)
input_layer = tf.keras.layers.Input(input_shape)

#create the base layers
base_layers = layers.experimental.preprocessing.Rescaling(1./255, name='bl_1')(input_layer)
base_layers = layers.Conv2D(16, 3, padding='same', activation='relu', name='bl_2')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_3')(base_layers)
base_layers = layers.Conv2D(32, 3, padding='same', activation='relu', name='bl_4')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_5')(base_layers)
base_layers = layers.Conv2D(64, 3, padding='same', activation='relu', name='bl_6')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_7')(base_layers)
base_layers = layers.Flatten(name='bl_8')(base_layers)

In [ ]:
input_layer

<KerasTensor: shape=(None, 400, 350, 3) dtype=float32 (created by layer 'input_1')>

In [21]:
#create the classifier branch
classifier_branch = layers.Dense(128, activation='relu', name='cl_1')(base_layers)
classifier_branch = layers.Dense(num_classes, name='cl_head')(classifier_branch) 

In [22]:
#create the localiser branch
locator_branch = layers.Dense(128, activation='relu', name='bb_1')(base_layers)
locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(6, activation='sigmoid', name='bb_head')(locator_branch)

In [ ]:
locator_branch

<KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'bb_head')>

In [23]:
model = tf.keras.Model(input_layer,
           outputs=[classifier_branch,locator_branch])


In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 bl_1 (Rescaling)               (None, 400, 350, 3)  0           ['input_1[0][0]']                
                                                                                                  
 bl_2 (Conv2D)                  (None, 400, 350, 16  448         ['bl_1[0][0]']                   
                                )                                                                 
                                                                                              

In [25]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, v):
        if not isinstance(v, Vector):
            return NotImplemented
        return Vector(self.x + v.x, self.y + v.y)

    def __sub__(self, v):
        if not isinstance(v, Vector):
            return NotImplemented
        return Vector(self.x - v.x, self.y - v.y)

    def cross(self, v):
        if not isinstance(v, Vector):
            return NotImplemented
        return self.x*v.y - self.y*v.x



In [26]:
class Line:
    # ax + by + c = 0
    def __init__(self, v1, v2):
        self.a = v2.y - v1.y
        self.b = v1.x - v2.x
        self.c = v2.cross(v1)

    def __call__(self, p):
        return self.a*p.x + self.b*p.y + self.c

    def intersection(self, other):
        # See e.g.     https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection#Using_homogeneous_coordinates
        if not isinstance(other, Line):
            return NotImplemented
        w = self.a*other.b - self.b*other.a
        return Vector(
            (self.b*other.c - self.c*other.b)/w,
            (self.c*other.a - self.a*other.c)/w
        )


In [27]:
def rectangle_vertices(cx, cy, w, h, cos, sin):
    #angle = pi*r/180
    dx = w/2
    dy = h/2
    dxcos = dx * cos
    dxsin = dx * sin
    dycos = dy * cos
    dysin = dy * sin
    return (
        Vector(cx, cy) + Vector(-dxcos - -dysin, -dxsin + -dycos),
        Vector(cx, cy) + Vector( dxcos - -dysin,  dxsin + -dycos),
        Vector(cx, cy) + Vector( dxcos -  dysin,  dxsin +  dycos),
        Vector(cx, cy) + Vector(-dxcos -  dysin, -dxsin +  dycos)
    )

In [83]:
def intersection_area(r1, r2):
    # r1 and r2 are in (center, width, height, cos, sin) representation
    # First convert these into a sequence of vertices

    rect1 = rectangle_vertices(r1[0],r1[1],r1[2],r1[3],r1[4],r1[5])
    rect2 = rectangle_vertices(r2[0],r2[1],r2[2],r2[3],r2[4],r2[5])
    
    P_rect1 = (float( r1[2]) * (r1[3]))
    P_rect2 = (float( r2[2]) * (r2[3]))
    
    # Use the vertices of the first rectangle as
    # starting vertices of the intersection polygon.
    intersection = rect1

    # Loop over the edges of the second rectangle
    for p, q in zip(rect2, rect2[1:] + rect2[:1]):
        if len(intersection) <= 2:
            break # No intersection

        line = Line(p, q)
        
        # Any point p with line(p) <= 0 is on the "inside" (or on the boundary),
        # any point p with line(p) > 0 is on the "outside".

        # Loop over the edges of the intersection polygon,
        # and determine which part is inside and which is outside.
        new_intersection = []
        line_values = [line(t) for t in intersection]
        for s, t, s_value, t_value in zip(
            intersection, intersection[1:] + intersection[:1],
            line_values, line_values[1:] + line_values[:1]):
            t2 = print('*')
            tf.cond(s_value  <= 0, lambda:new_intersection.append(s),false_fn=lambda:t2)
            #if s_value  <= 0:
             #   new_intersection.append(s)
            tf.cond(s_value * t_value < 0, lambda: new_intersection.append(line.intersection(Line(s, t))),false_fn=lambda:t2)
            
            #if s_value * t_value < 0:
                # Points are on opposite sides.
                # Add the intersection of the lines to new_intersection.
             #   intersection_point = line.intersection(Line(s, t))
              #  new_intersection.append(intersection_point)

        intersection = new_intersection

    # Calculate area
    if len(intersection) <= 2:
        return 0
    
    inter=0.5 * sum(p.x*q.y - p.y*q.x for p, q in
                     zip(intersection, intersection[1:] + intersection[:1]))
    
    iou = (inter / float(P_rect1 + P_rect2 - inter)) 
    
    return iou

In [84]:
#losses = {"cl_head":tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), "bb_head":tfa.losses.GIoULoss}
losses = {"cl_head":tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), "bb_head":intersection_area}

In [85]:
model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])

In [86]:
trainTargets = {
    "cl_head": train_labels,
    "bb_head": train_targets
}
validationTargets = {
    "cl_head": validation_labels,
    "bb_head": validation_targets
}

In [87]:
history = model.fit(train_images, trainTargets,
             validation_data=(validation_images, validationTargets),
             batch_size=3,
             epochs=20,
             shuffle=True,
             verbose=1)


Epoch 1/20
*
*
*
*


InaccessibleTensorError: ignored

In [ ]:
a=model.predict(validation_images)

In [ ]:
a[1][1]
 

In [ ]:
b=validationTargets['bb_head']
b[1]

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import math
import numpy.matlib as npm

In [ ]:
def convert5Pointto8Point(cx_, cy_, w_, h_, cos_, sin_):

    #theta = math.radians(a_)
    bbox = npm.repmat([[cx_], [cy_]], 1, 5) + \
       np.matmul([[cos_, sin_],
                  [-sin_, cos_]],
                 [[-w_ / 2, w_/ 2, w_ / 2, -w_ / 2, w_ / 2 + 8],
                  [-h_ / 2, -h_ / 2, h_ / 2, h_ / 2, 0]])
    # add first point
    x1, y1 = bbox[0][0], bbox[1][0]
    # add second point
    x2, y2 = bbox[0][1], bbox[1][1]
    # add third point
    #x3, y3 = bbox[0][4], bbox[1][4]   
    # add forth point
    x3, y3 = bbox[0][2], bbox[1][2]
    # add fifth point
    x4, y4 = bbox[0][3], bbox[1][3]

    return [x1, y1, x2, y2, x3, y3, x4, y4]

In [ ]:
IMAGE_PATH = os.path.join(paths['Images'], 'Validation', '/content/Images/Validation/2022_02_26_16_19_36.jpg')

In [ ]:
i=3

Ground_truth=b[i]

Predict=a[1][i]




In [ ]:
img = Image.open(IMAGE_PATH)
#img = Image.new('L', (width, height), 0)
polygon = convert5Pointto8Point(np.ceil(Ground_truth[0]*350), np.ceil(Ground_truth[1]*400), np.ceil(Ground_truth[2]*350), np.ceil(Ground_truth[3]*400), -Ground_truth[4], Ground_truth[5]) 
ImageDraw.Draw(img).polygon(polygon, outline='blue')   
polygon = convert5Pointto8Point(np.ceil(Predict[0]*350), np.ceil(Predict[1]*400), np.ceil(Predict[2]*350), np.ceil(Predict[3]*400), -Predict[4], Predict[5]) 
ImageDraw.Draw(img).polygon(polygon, outline='red')        
plt.imshow(img)
plt.show()